In [21]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from lsst.sims.photUtils import Sed
from lsst.sims.photUtils import Bandpass
from lsst.sims.photUtils import PhotometricParameters

import os

In [8]:
# Let's find the full well magnitude per filter for a variety of exposure times.
filters = ['u', 'g', 'r', 'i', 'z', 'y']



In [10]:
band_dir = os.getenv("LSST_THROUGHPUTS_BASELINE")
bps = []

for filtername in filters:
    
    tempB = Bandpass()
    tempB.readThroughput(band_dir+'filter_'+filtername+'.dat')
    bps.append(tempB)

In [11]:
bps

In [12]:
wave = np.arange(bps[0].wavelen.min(), bps[-1].wavelen.max()+1)

In [55]:
const = 1e-5
flat_sed = Sed(wavelen=wave, fnu=np.zeros(wave.size, dtype=float) + const)

In [56]:
flat_sed.calcMag(bps[3])

21.400065622282231

In [57]:
params = PhotometricParameters(exptime=15, nexp=1)

In [58]:
flat_sed.calcADU(bps[0], params) 

5899.3425879503184

In [46]:
params.gain  # Electrons per ADU

2.3

In [61]:
def electrons_in_peak(adu, fwhm_eff, gain, platescale):
    # Given total number of ADUs from a star, and seeing,
    # return the 
    c = fwhm_eff/2.3548/platescale
    peak = adu/c/np.sqrt(2.*np.pi)
    peak = peak*gain
    return peak

In [63]:
# OK, let's make arrays of mags and peak electron counts.
electrons_in_peak(flat_sed.calcADU(bps[1], params) , 1., params.gain, params.platescale)


4073.7081229183023